In [63]:
# @title imports
import numpy as np
import pandas as pd
import cv2
import matplotlib.pyplot as plt

import json
import os
import random
import collections
!pip install imantics


import imantics
from PIL import Image
from skimage.transform import resize

from sklearn.cluster import KMeans
from sklearn.model_selection import train_test_split
from collections import Counter
from scipy.ndimage import label

import tensorflow as tf
import tensorflow_hub as hub
from tensorflow.keras import layers
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D,Concatenate, Dropout, Conv2DTranspose, concatenate, AveragePooling2D,Activation,UpSampling2D,BatchNormalization
from tensorflow.keras.losses import CategoricalCrossentropy
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.utils import to_categorical

%matplotlib inline

In [64]:
#  @title Load the dataset

images_path = '/content/drive/MyDrive/ASDC_dataset/images'
labels_path = '/content/drive/MyDrive/ASDC_dataset/COCO_Football Pixel.json'


In [65]:
# #  @title Load images and masks

# image_size = 512
# input_image_size = (1920, 1080)

# def read_image(path):
#     print(f"Reading image from path: {path}")
#     img = cv2.imread(path)
#     img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
#     img = cv2.resize(img, (image_size, image_size))
#     return img

# annote = json.load(open(labels_path))

# id_to_images = {image['id']:image['file_name'] for image in annote['images']}

# read_image(images_path)

In [ ]:
image_files = os.listdir(images_path)

# @title images
images = np.zeros((len(image_files), image_size, image_size, 3), dtype=np.uint8)

for i, image_filename in enumerate(image_files):
    cur_image = read_image(os.path.join(images_path, image_filename))
    images[i] = cur_image


In [67]:
# @title masks
masks = np.zeros((len(image_files), image_size, image_size, 1), dtype=bool)

for annotation in annote["annotations"]:
    image_id = annotation["image_id"]
    segmentation = annotation["segmentation"]

    cur_mask = imantics.Polygons(segmentation).mask(*input_image_size).array
    cur_mask = np.expand_dims(resize(cur_mask, (image_size, image_size), mode='constant', preserve_range=True), 2)

    mask_index = image_id - 1

    # Ensure mask_index is within bounds
    if 0 <= mask_index < len(image_files):
        masks[mask_index] = masks[mask_index] | cur_mask
    else:
        print(f"Invalid image_id: {image_id}")

In [68]:
masked_images = []
for i in range(len(masks)):
    temp_mask = np.copy(masks[i])
    temp_mask = temp_mask.astype(np.uint8)
    temp_image = np.copy(images[i])
    temp_masked_image = cv2.bitwise_and(temp_image, temp_image, mask=temp_mask)
    masked_images.append(temp_masked_image)

In [69]:
# Define the target color and threshold for each team
team_a_color = np.array([129, 239, 252])
team_b_color = np.array([170, 25, 50])
ref_color = np.array([30, 50, 90])

# The distance from the center color for the team
distance_a = 60
distance_b = 60
distance_ref = 30

# The team color in the multilass mask
class_a_color = 200
class_b_color = 150
class_ref_color = 40

In [70]:
# Create inverted masks
inverted_masks = []
for i in range(len(masks)):
    temp_inv = masks[i].copy()
    temp_inv =  temp_inv.astype(np.uint8)
    temp_inv[temp_inv == 0 ] = 255
    temp_inv[temp_inv == 1 ] = 0
    inverted_masks.append(temp_inv)

In [71]:
def extractTeamColor(image, target_color, threshold, class_color):
    # Calculate the euclidean distance between each pixel and the target color
    color_distance = np.linalg.norm(image - target_color, axis=2)
    # Create a mask to filter pixels close to the target color
    color_mask = color_distance < threshold

    # Apply the mask to the original image
    extracted_image = np.zeros_like(image)
    extracted_image[color_mask] = image[color_mask]

    # Assign the class color to each channel of the extracted image
    for i in range(extracted_image.shape[2]):
        extracted_image[..., i][extracted_image[..., i] != 0] = class_color

    return extracted_image

In [72]:
def recolorWithMask(image):
    dark_gray = [10, 10, 10]
    black = [0, 0, 0]

    # Create a mask for black pixels
    black_pixels_mask = np.all(image == black, axis=2)

    # Create a mask for white pixels
    white_pixels_mask = np.all(image == [255, 255, 255], axis=2)

    # Apply black color to the black pixels
    image[black_pixels_mask] = dark_gray

    # Apply black color to the white pixels
    image[white_pixels_mask] = black

    return image

def recolorFinalClassesMask(image):
    red = [255, 0, 0]
    blue = [0, 0, 255]
    green = [0, 255, 0]
    black = [0, 0, 0]
    yellow = [255, 255, 0]

    #-----------------------
    team_a = [200, 200, 200]
    team_b  =[150, 150, 150]
    #ref_color = [40, 40, 40]
    #====
    comb1 =[94, 94, 94]
    '''comb2 = [240, 240, 240]
    comb3 = [140, 140, 140]
    comb4 = [136, 136, 136]'''

    teamA_pixels_mask = np.all(image == team_a, axis=2)
    teamB_pixels_mask = np.all(image == team_b, axis=2)
    other_pixels_mask = np.all(image == comb1, axis=2)
    #otherMask = np.all((image == comb2) or(image == comb1)  , axis=2)

    '''colors = [comb1, comb2, comb3, comb4]
    combined_mask = np.zeros(image.shape[:2], dtype=bool)

    for color in colors:
        mask = np.all(image == color, axis=2)
        combined_mask = np.logical_or(combined_mask, mask)'''

    image[teamA_pixels_mask] = blue
    image[teamB_pixels_mask] = red
    #image[ref_pixels_mask] = green
    image[other_pixels_mask] = black

    return image

In [73]:
def createClassMask(image, team_color):
    # Find connected regions
    labels, num_features = label(np.any(image != 0, axis=-1))
    #print('labels', labels[90])
    # Loop through each connected region
    for i in range(1, num_features + 1):
        mask = (labels == i)
        #print('mask', mask.shape)
        # Get the colors in the region
        region_colors = image[mask]

        flat_array = region_colors.flatten()
        #print(flat_array)
        vals, counts = np.unique(flat_array, return_counts=True)
        for val, count in zip(vals, counts):
            if val == team_color  and count > 150:
                image[mask] = team_color
            elif (val == team_color  and count < 150) or len(vals) == 1:
                image[mask] = 0

    return image

In [ ]:
team_a_masks = []
team_b_masks = []
final_masks = []

for i in range(len(masked_images)):
    extracted_color_A = extractTeamColor(masked_images[i], team_a_color, distance_a, class_a_color)
    new_im_a = extracted_color_A + inverted_masks[i]
    recolor_a = recolorWithMask(new_im_a.copy())
    team_a_mask = createClassMask(recolor_a.copy(),class_a_color)
    team_a_masks.append(team_a_mask)
    #----
    extracted_color_B = extractTeamColor(masked_images[i], team_b_color, distance_b, class_b_color)
    new_im_b = extracted_color_B + inverted_masks[i]
    recolor_b = recolorWithMask(new_im_b.copy())
    team_b_mask = createClassMask(recolor_b.copy(),class_b_color)
    team_b_masks.append(team_b_mask)
     #----
    '''extracted_color_ref = extractTeamColor(masked_images[i], ref_color, distance_ref, class_ref_color)
    new_im_ref = extracted_color_ref + inverted_masks[i]
    recolor_ref = recolorWithMask(new_im_ref.copy())
    team_ref_mask = createClassMask(recolor_ref.copy(), class_ref_color)'''

    final_mask = team_a_mask + team_b_mask # + team_ref_mask
    final_masks.append(final_mask)

In [ ]:
extracted_color_c = extractTeamColor(masked_images[0], team_a_color, distance_a, class_a_color)
new_im_c = extracted_color_c + inverted_masks[i]
recolor_c = recolorWithMask(new_im_c.copy())
team_c_mask = createClassMask(recolor_c.copy(),class_a_color)
team_c_mask2 =  createClassMask(team_c_mask.copy(),class_a_color)

In [ ]:
for i in range(len(masked_images)):
    final_masks[i] = recolorFinalClassesMask(final_masks[i] )

In [ ]:
plt.imshow(final_masks[0])

In [ ]:
# @title preprocessing data
def encode_rgb_masks(rgb_masks):
    gray_masks = np.zeros((rgb_masks.shape[0], rgb_masks.shape[1]), dtype=np.uint8)

    # Define color-to-intensity mappings for each class
    color_to_intensity = {
        (0, 0, 0): 0,
        (255, 0, 0): 1,
        (0, 0, 255): 2
    }
    for i in range(rgb_masks.shape[0]):
        for j in range(rgb_masks.shape[1]):
            color = tuple(rgb_masks[i, j])
            gray_masks[i, j] = color_to_intensity.get(color, 0)

    return gray_masks

In [ ]:
# @title Normalization

num_classes = 3

encoded_masks= []
for i in range(len(final_masks)):
    encoded_masks.append(encode_rgb_masks(final_masks[i]))

input_images = np.array(images)
input_masks = np.array(encoded_masks)

In [ ]:

def convert_rgb_to_grayscale(rgb_image):
    return np.dot(rgb_image[...,:3], [0.2989, 0.5870, 0.1140])

input_images = input_images / 255.0

In [ ]:
# @title splitting into training and testing
split_ratio = 0.7
split_index = int(len(input_images) * split_ratio)

train_images = input_images[:split_index]
train_masks = input_masks[:split_index]

val_images = input_images[split_index:]
val_masks = input_masks[split_index:]

split_ratio2 = 0.5
split_index2 = int(len(val_images) * split_ratio)

print(split_index2)
validation_images = val_images[:split_index2]
validation_masks = val_masks[:split_index2]

test_images = val_images[split_index2:]
test_masks = val_masks[split_index2:]

In [ ]:
# Convert grayscale masks to one-hot encoded format
num_classes = 3
train_masks_encoded = np.zeros((len(train_masks), 512, 512, num_classes), dtype=np.float32)
validation_masks_encoded = np.zeros((len(validation_masks), 512, 512, num_classes), dtype=np.float32)

test_masks_encoded = np.zeros((len(test_masks), 512, 512, num_classes), dtype=np.float32)

for i in range(len(train_masks)):
    train_masks_encoded[i] = tf.one_hot(train_masks[i], num_classes)

for i in range(len(validation_masks)):
    validation_masks_encoded[i] = tf.one_hot(validation_masks[i], num_classes)

for i in range(len(test_masks)):
    test_masks_encoded[i] = tf.one_hot(test_masks[i], num_classes)

In [ ]:
plt.imshow(input_images[0])
plt.imshow(train_masks_encoded[0])

In [ ]:
train_masks_encoded[0].shape

In [ ]:
# @title UNet model
def conv_block(input, num_filters):
    conv = tf.keras.layers.Conv2D(num_filters, 3, padding="same")(input)
    conv = tf.keras.layers.BatchNormalization()(conv)
    conv = tf.keras.layers.Activation("relu")(conv)
    conv = tf.keras.layers.Conv2D(num_filters, 3, padding="same")(conv)
    conv = tf.keras.layers.BatchNormalization()(conv)
    conv = tf.keras.layers.Activation("relu")(conv)
    return conv

def encoder_block(input, num_filters):
    skip = conv_block(input, num_filters)
    pool = tf.keras.layers.MaxPool2D((2, 2))(skip)
    return skip, pool

def decoder_block(input, skip, num_filters):
    up_conv = tf.keras.layers.Conv2DTranspose(num_filters, (2, 2), strides=2, padding="same")(input)
    conv = tf.keras.layers.Concatenate()([up_conv, skip])
    conv = conv_block(conv, num_filters)
    return conv

def unet_model(input_shape, num_classes=3):
    inputs = tf.keras.layers.Input(input_shape)

    skip1, pool1 = encoder_block(inputs, 64)
    skip2, pool2 = encoder_block(pool1, 128)
    skip3, pool3 = encoder_block(pool2, 256)
    skip4, pool4 = encoder_block(pool3, 512)

    bridge = conv_block(pool4, 1024)
     decode1 = decoder_block(bridge, skip4, 512)
    decode2 = decoder_block(decode1, skip3, 256)
    decode3 = decoder_block(decode2, skip2, 128)
    decode4 = decoder_block(decode3, skip1, 64)

    outputs = tf.keras.layers.Conv2D(num_classes, 1, padding="same", activation="softmax")(decode4)

    model = tf.keras.Model(inputs=inputs, outputs=outputs)
    return model

In [ ]:
model = unet_model((512, 512, 3))
max_norm = 1.0

optimizer = tf.keras.optimizers.Adam(clipvalue=max_norm)
model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
epochs = 5
batch_size = 4
unet_history = model.fit(train_images, train_masks_encoded, validation_data=(validation_images, validation_masks_encoded),
          batch_size=batch_size, epochs=epochs)

In [ ]:
model.save('unet1-football-asma.h5')

In [ ]:
plt.plot(unet_history.history["accuracy"])
plt.plot(unet_history.history["val_accuracy"])

In [ ]:
plt.plot(unet_history.history["loss"])
plt.plot(unet_history.history["val_loss"])
plt.ylabel('Cross Entropy')
plt.title('Loss')
plt.xlabel('epoch')
plt.legend(('training loss','val_loss'),loc='upper right')

In [ ]:
validation_loss, validation_accuracy = model.evaluate(validation_images, validation_masks_encoded, batch_size = 4)

In [ ]:
test_loss, test_accuracy = model.evaluate(test_images, test_masks_encoded, batch_size = 4)

In [ ]:
train_accuracy = unet_history.history['accuracy'][-1]
print(f'Model Accuracy on the Training Dataset: {round(train_accuracy * 100, 2)}%')
print(f'Model Accuracy on the Validation Dataset: {round(validation_accuracy * 100, 2)}%')
print(f'Model Accuracy on the Test Dataset: {round(test_accuracy * 100, 2)}%')

In [ ]:
predicted_mask = model.predict(test_images[0])

In [ ]:
unet_predict = model.predict(test_images)

In [ ]:
def show_result(og, unet,  target):
    fig, axs = plt.subplots(1, 3, figsize=(10,5))
    axs[0].set_title("Original")
    axs[0].imshow(og)
    axs[0].axis('off')

    axs[1].set_title("U-Net Predicted Mask")
    axs[1].imshow(unet)
    axs[1].axis('off')

    axs[2].set_title("Ground Truth")
    axs[2].imshow(target)
    axs[2].axis('off')

In [ ]:
show_test_idx = random.sample(range(47), 3)
for idx in show_test_idx:
    show_result(test_images[idx], unet_predict[idx], test_masks[idx])

In [ ]:
show_test_idx

In [ ]:
# @title Field Lines extraction

def detectLinesCircles(image):
    hsv = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)
    lower_green = np.array([35, 50, 50])
    upper_green = np.array([90, 255, 255])

    mask = cv2.inRange(hsv, lower_green, upper_green)
    blurred_mask = cv2.GaussianBlur(mask, (5, 5), 0)

    edges = cv2.Canny(mask, 50, 150)

    lines = cv2.HoughLines(edges, 1, np.pi / 180, threshold=100)

    side_list = []
    for line in lines:
        rho, theta = line[0]

        slope = np.tan(theta)

        if slope < -3 and slope > -100:
            side = "Left"
        elif slope > 3 and slope < 100:
           side = "Right"
        elif (slope<0.5 and slope>- 0.5) or slope < -100 or slope > 100:
            side = "Center"
        else:
            if slope<0:
                side = 'Right'
            else:
                side = 'Left'
        #print(side)
        #print('slope ', slope)
        side_list.append(side)

        a = np.cos(theta)
        b = np.sin(theta)
        x0 = a * rho
        y0 = b * rho
        x1 = int(x0 + 1000 * (-b))
        y1 = int(y0 + 1000 * (a))
        x2 = int(x0 - 1000 * (-b))
        y2 = int(y0 - 1000 * (a))
        cv2.line(image, (x1, y1), (x2, y2), (0, 255, 0), 2)

        # Hough circle transform to detect circles
    '''circles = cv2.HoughCircles(blurred_mask, cv2.HOUGH_GRADIENT, dp=0.1, minDist=150, param1=60, param2=30, minRadius=0, maxRadius=0)

    if circles is not None:
        circles = np.round(circles[0, :]).astype(int)

        filtered_circles = []
        for (x, y, r) in circles:
            if mask[y, x] != 0:
                filtered_circles.append((x, y, r))

        for (x, y, r) in filtered_circles:
            cv2.circle(image, (x, y), r, (0, 255, 0), 2)'''

    isCenter = any(item in 'Center' for item in side_list)

    if isCenter:
        field_side = 'Center'
         else:
        sorted_lines = [item for items, c in Counter(side_list).most_common()
                                  for item in [items] * c]
        field_side =  sorted_lines[0]

    return image, field_side

In [ ]:
id = 400
test_img = images[id].copy()
test_img, field_side = detectLinesCircles(test_img)

field_side

In [ ]:
plt.imshow(test_img)